In [740]:
import re
import json
import hashlib
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime
from difflib import SequenceMatcher

In [2]:
import utils.db as db

In [279]:
def clean_text(text_str):
    return text_str.strip().replace("\n", "").replace("\r", " ").replace("  ", " ")

In [526]:
conn = db.connect_mongo_db("senate-publication")

In [50]:
select_columns = [
    "summary",
    "type",
    "authors",
    "parties",
    "url",
    "date",
    "normalized_summary_v2",
    "topics_normalized_text_12_model",
]

In [528]:
publications_cursor = conn.publication.find()
publications_df = pd.DataFrame(publications_cursor)
publications_df.head()

,_id,type,date,url,summary,authors,parties,full_text,normalized_summary,normalized_summary_v2,summary_adj_noun,summary_adj_only,summary_nouns_only,topics_normalized_text_12_model,doc_url,doc_path
0,f02a7c92756ec00e5d112766d3d92214,iniciativas,2022-09-27,https://www.senado.gob.mx/64/gaceta_del_senado...,Propone actualizar las referencias al salario ...,[Noé Fernando Castañón Ramírez],[MC],C. Alejandro Armenta Mier\nPresidente de la Me...,proponer actualizar referencia salario mínimo ...,actualizar referencia salario mínimo unidad ac...,referencia salario mínimo unidad actualización...,salario mínimo vigente sujetos funcionario,referencia unidad actualización ciudad sanción,"[{'topic': 0, 'score': 0.14669232070446014}, {...",NaN,NaN
1,b17a1dd8f043ceeb993269ca9971af51,iniciativas,2022-09-20,https://www.senado.gob.mx/64/gaceta_del_senado...,Propone establecer que la educación impartida ...,"[Martha Cecilia Márquez Alvarado, Joel Padilla...",[PT],Página 1 de 12 \nDe las senadoras Geovanna del...,proponer establecer educación impartido inculc...,educación impartido inculcar concepto principi...,educación impartido concepto principio relacio...,impartido principio relacionado cuidado medio ...,educación concepto ambiente expresión solidari...,"[{'topic': 1, 'score': 0.3063770830631256}, {'...",https://infosen.senado.gob.mx/sgsp/gaceta/65/2...,/Users/andreamarin/Documents/personal_projects...
2,ee1822112771d6044275c427a7a8a2d7,iniciativas,2022-09-20,https://www.senado.gob.mx/64/gaceta_del_senado...,Propone establecer que el registro civil en Of...,[Lilia Margarita Valdez Martínez],[MORENA],I'Dra. Jvlargaríta \n1 Ya[áez + \nSenadora \nI...,proponer establecer registro civil oficialía p...,registro civil oficialía población indígén con...,registro civil población traductor lengua regi...,civil traductor deber garantizarl derecho orig...,registro población lengua registro campaña vez...,"[{'topic': 0, 'score': 0.4472990036010742}, {'...",https://infosen.senado.gob.mx/sgsp/gaceta/65/2...,/Users/andreamarin/Documents/personal_projects...
3,6e9aef9142845000afaed04cfbaf6bd4,iniciativas,2022-09-27,https://www.senado.gob.mx/64/gaceta_del_senado...,Propone atribuir a la Secretaría de Bienestar ...,"[Martha Cecilia Márquez Alvarado, Joel Padilla...",[PT],SENADORAS \nY SENADORES \nTRABAJANDO \nDe las ...,proponer atribuir secretaría bienestar función...,atribuir bienestar función elaborar política p...,bienestar función política público pleno ejerc...,público pleno derecho pepenadora,bienestar función política ejercicio desarroll...,"[{'topic': 0, 'score': 0.3251364827156067}, {'...",https://infosen.senado.gob.mx/sgsp/gaceta/65/2...,/Users/andreamarin/Documents/personal_projects...
4,c47e1c3be5449bc180c17af4a8b3ca3f,iniciativas,2022-09-27,https://www.senado.gob.mx/64/gaceta_del_senado...,Propone establecer que la Secretaría de Desarr...,[Martha Cecilia Márquez Alvarado],[PT],Martha Cecilia Márquez Alvarado \nSENADORA DE ...,proponer establecer secretaría desarrollo soci...,desarrollo social fomentar creación estancia c...,desarrollo social creación estancia cuidado pe...,social cuidado adulto mayor necesario adecuado...,desarrollo creación estancia persona condición...,"[{'topic': 1, 'score': 0.11103018373250961}, {...",https://infosen.senado.gob.mx/sgsp/gaceta/65/2...,/Users/andreamarin/Documents/personal_projects...


In [533]:
publications_df["summary"] = publications_df.summary.apply(clean_text)

In [698]:
publications_df["type"].unique()

array(['iniciativas', 'proposiciones'], dtype=object)

## Explorar datos abiertos

In [155]:
legis = ["64", "65"]

In [706]:
iniciativas_og = None
for legis_num in legis:
    with open(f"./data/iniciativa_{legis_num}.json", "r") as f:
        json_data = json.load(f)
        
    legis_data = pd.DataFrame(json_data)
    if iniciativas_og is None:
        iniciativas_og = legis_data
    else:
        iniciativas_og = pd.concat([iniciativas_og, legis_data])

iniciativas_og.shape

(5612, 14)

In [707]:
proposiciones_og = None
for legis_num in legis:
    with open(f"./data/proposicion_{legis_num}.json", "r") as f:
        json_data = json.load(f)
        
    legis_data = pd.DataFrame(json_data)
    
    if proposiciones_og is None:
        proposiciones_og = legis_data
    else:
        proposiciones_og = pd.concat([proposiciones_og, legis_data])

proposiciones_og.shape

(4274, 14)

In [708]:
iniciativas_og.head(1)

,id,titulo,fecha_presentacion,sentido_dictamen,url_gaceta,estado,sintesis,legislatura,senadores,comisiones,leyes_modifica,tipo,camara_origen,fecha_aprobacion
0,766954,Proyecto de decreto por el que se adiciona una...,2021-08-25,No Aplica,120002,Pendiente,Propone crear los certificados de vacunación e...,64,Geovanna del Carmen Bañuelos de La Torre (PT)<br>,Salud (Coordinadora) <br>Estudios Legislativos...,Ley General de Salud<br>,iniciativa,No Aplica,0000-00-00


In [709]:
proposiciones_og.head(1)

,id,titulo,fecha_presentacion,sentido_dictamen,url_gaceta,estado,sintesis,legislatura,senadores,comisiones,leyes_modifica,tipo,camara_origen,fecha_aprobacion
0,769584,Punto de acuerdo por el que se exhorta al titu...,2021-08-25,No Aplica,117641,Desechada,PRIMERO. La Comisión Permanente del H. Congres...,64,Senadores integrantes del PAN (PAN)<br>,Gobernación (Coordinadora) <br>,No Alpica,proposicion,No Aplica,2022-11-04


In [744]:
iniciativas = iniciativas_og.rename(
    columns = {
        "url_gaceta": "url_id",
        "comisiones": "commissions",
        "leyes_modifica": "modified_laws",
        "titulo": "title",
        "estado": "status",
        "legislatura": "legislature",
        "fecha_aprobacion": "approval_date",
        "sintesis": "da_summary"
    }
)
iniciativas = iniciativas[[
    "url_id",
    "commissions",
    "modified_laws",
    "title",
    "status",
    "legislature",
    "approval_date",
    "da_summary"
]]
iniciativas["type"] = "iniciativas"

In [745]:
proposiciones = proposiciones_og.rename(
    columns = {
        "url_gaceta": "url_id",
        "comisiones": "commissions",
        "titulo": "title",
        "estado": "status",
        "legislatura": "legislature",
        "fecha_aprobacion": "approval_date",
        "sintesis": "da_summary"
    }
)
proposiciones = proposiciones[[
    "url_id",
    "commissions",
    "title",
    "status",
    "legislature",
    "approval_date",
    "da_summary"
]]
proposiciones["type"] = "proposiciones"

In [868]:
publications_datos_abiertos = pd.concat([
    iniciativas, proposiciones
]).reset_index(drop=True)

publications_datos_abiertos = publications_datos_abiertos[
    publications_datos_abiertos.url_id != ""
]

publications_datos_abiertos.shape

(9881, 9)

In [869]:
publications_datos_abiertos["da_summary"] = publications_datos_abiertos.da_summary.apply(clean_text)

In [870]:
# remove true duplicates
publications_datos_abiertos = publications_datos_abiertos.drop_duplicates()

# hardcoded duplicate
publications_datos_abiertos = publications_datos_abiertos.drop([4608, 4677])

publications_datos_abiertos = publications_datos_abiertos.reset_index(drop=True)

publications_datos_abiertos.shape

(9874, 9)

In [872]:
publications_datos_abiertos.url_id.nunique()

9868

In [873]:
publications_datos_abiertos.loc[publications_datos_abiertos.duplicated(subset=["url_id", "type"], keep=False)].shape

(12, 9)

In [874]:
publications_datos_abiertos.loc[
    publications_datos_abiertos.duplicated(subset=["url_id", "type"], keep=False)
].sort_values(by=["url_id"])

,url_id,commissions,modified_laws,title,status,legislature,approval_date,da_summary,type
2420,100850,Hacienda y Crédito Público (Coordinadora) <br>...,Ley de Protección y Defensa al Usuario de Serv...,Proyecto de decreto por el que se reforman y a...,Pendiente,64,0000-00-00,Propone eliminar las referencias realizadas a ...,iniciativas
2421,100850,Educación (Coordinadora) <br>Estudios Legislat...,Ley General de Educación<br>Ley Orgánica de la...,Proyecto de decreto por el que se reforman div...,Pendiente,64,0000-00-00,Propone homologar el término actual de Secreta...,iniciativas
1323,109567,Puntos Constitucionales (Coordinadora) <br>Est...,Constitución Política de los Estados Unidos Me...,Proyecto de decreto por el que se reforma el p...,Pendiente,64,0000-00-00,Propone que en el caso de los delitos establec...,iniciativas
1324,109567,Justicia (Coordinadora) <br>Estudios Legislati...,Ley Nacional de Ejecución Penal<br>,Proyecto de decreto por el que se adiciona un ...,Pendiente,64,0000-00-00,Propone que el Juez de Ejecución tenga a su ca...,iniciativas
4978,124713,"Medio Ambiente, Recursos Naturales y Cambio Cl...",Expedición de Nueva Ley<br>,Proyecto de decreto por el que se expide la Le...,Pendiente,65,0000-00-00,Propone crear un nuevo ordenamiento jurídico q...,iniciativas
4979,124713,Puntos Constitucionales (Coordinadora) <br>Est...,Constitución Política de los Estados Unidos Me...,Proyecto de decreto por el que se reforma el a...,Pendiente,65,0000-00-00,Propone establecer que las personas tendrán de...,iniciativas
4298,128941,Estudios Legislativos (Coordinadora) <br>Punto...,Constitución Política de los Estados Unidos Me...,Proyecto de decreto por el que se reforma la f...,Pendiente,65,0000-00-00,Propone establecer que los partidos políticos ...,iniciativas
4598,128941,Puntos Constitucionales (Coordinadora) <br>Est...,Constitución Política de los Estados Unidos Me...,Proyecto de decreto por el que se reforma la f...,Pendiente,65,0000-00-00,Propone establecer que los partidos políticos ...,iniciativas
4473,129747,Salud (Coordinadora) <br>Estudios Legislativos...,Ley General de Salud<br>,Proyecto de decreto por el que se adiciona una...,Pendiente,65,0000-00-00,Propone que se establezca e implemente un Prog...,iniciativas
4507,129747,"Desarrollo Urbano, Ordenamiento Territorial y ...",Ley Orgánica de la Administración Pública Fede...,Proyecto de decreto por el que se reforma la f...,Pendiente,65,0000-00-00,"Propone que, a la Secretaría de Desarrollo Agr...",iniciativas


In [731]:
iniciativas_og.loc[
    iniciativas_og.url_gaceta == "100850"
]

,id,titulo,fecha_presentacion,sentido_dictamen,url_gaceta,estado,sintesis,legislatura,senadores,comisiones,leyes_modifica,tipo,camara_origen,fecha_aprobacion
2421,764524,Proyecto de decreto por el que se reforman y a...,2019-11-14,No Aplica,100850,Pendiente,Propone eliminar las referencias realizadas a ...,64,Minerva Hernández Ramos (PAN)<br>,Hacienda y Crédito Público (Coordinadora) <br>...,Ley de Protección y Defensa al Usuario de Serv...,iniciativa,No Aplica,0000-00-00
2422,764525,Proyecto de decreto por el que se reforman div...,2019-11-14,No Aplica,100850,Pendiente,Propone homologar el término actual de Secreta...,64,Minerva Hernández Ramos (PAN)<br>,Educación (Coordinadora) <br>Estudios Legislat...,Ley General de Educación<br>Ley Orgánica de la...,iniciativa,No Aplica,0000-00-00


In [732]:
publications_df["url_id"] = publications_df.url.apply(
    lambda u: u.split("/")[-1]
)

In [733]:
publications_df.loc[
    (publications_df.type == "iniciativas")
    & (publications_df.url_id == "100850")
]

,_id,type,date,url,summary,authors,parties,full_text,normalized_summary,normalized_summary_v2,summary_adj_noun,summary_adj_only,summary_nouns_only,topics_normalized_text_12_model,doc_url,doc_path,url_id
9314,366678b3e334a9d4ccefbfc7f2abe039,iniciativas,2019-11-14,https://www.senado.gob.mx/65/gaceta_del_senado...,Propone eliminar las referencias realizadas a ...,[Minerva Hernández Ramos],[PAN],Iniciativa con proyecto de Decreto por la que ...,proponer eliminar referencia realizado socieda...,eliminar referencia realizado sociedad financi...,referencia realizado sociedad financiero objet...,realizado financiero múltiple regulado patrimo...,referencia sociedad objeto entidad vínculo tér...,"[{'topic': 0, 'score': 0.4201943278312683}, {'...",https://infosen.senado.gob.mx/sgsp/gaceta/64/2...,/Users/andreamarin/Documents/personal_projects...,100850


In [737]:
publications_df.loc[
    (publications_df.type == "iniciativas")
    & (publications_df.date == "2019-11-14")
].shape

(19, 17)

In [833]:
publications_datos_abiertos.head(1)

,url_id,commissions,modified_laws,title,status,legislature,approval_date,da_summary,type
0,120002,Salud (Coordinadora) <br>Estudios Legislativos...,Ley General de Salud<br>,Proyecto de decreto por el que se adiciona una...,Pendiente,64,0000-00-00,Propone crear los certificados de vacunación e...,iniciativas


In [834]:
publications_df.loc[
    ~publications_df.url_id.isin(publications_datos_abiertos.url_id)
].shape

(1093, 17)

In [835]:
publications_datos_abiertos.loc[
    ~publications_datos_abiertos.url_id.isin(publications_df.url_id)
].shape

(18, 9)

# Crear nuevas dbs

In [875]:
publications_df.shape

(10943, 17)

In [876]:
publications_df["_id"].nunique()

10943

In [885]:
complete_data = publications_df.merge(
    publications_datos_abiertos,
    on=["url_id", "type"],
    how="left"
)
complete_data.shape

(10949, 24)

## Limpiar datos

In [840]:
def get_match_ratio(row):
    return SequenceMatcher(None, row.summary, row.da_summary).ratio()

In [889]:
def split_commissions(commissions_str):
    
    if pd.isna(commissions_str):
        return None
    
    # remove extra information
    commissions_str = re.sub("\(.*?\)", "", commissions_str)
    
    # remove line breaks
    commissions_str = re.sub(" *<br> *", "####", commissions_str)
    
    # remove last appearance
    commissions_str = re.sub("####$", "", commissions_str)
    
    # remove extra words
    commissions_str = re.sub(" *(Primera|Segunda)$", "", commissions_str)
    
    return commissions_str.split("####")

In [603]:
def split_laws(laws_str):
    
    if pd.isna(laws_str):
        return None
    
    # remove line breaks
    laws_str = re.sub(" *<br> *", "####", laws_str)
    
    # remove last appearance
    laws_str = re.sub("####$", "", laws_str)
    
    return laws_str.split("####")

In [886]:
da_duplicates = complete_data.duplicated(subset=["_id"], keep=False)
da_duplicates.sum()

12

In [887]:
complete_data["match_ratio"] = 1.00
complete_data.loc[da_duplicates, "match_ratio"] = complete_data.loc[da_duplicates].apply(get_match_ratio, axis=1)

In [888]:
complete_data = complete_data.loc[complete_data.match_ratio == 1]
complete_data.shape

(10943, 25)

In [890]:
complete_data["commissions"] = complete_data.commissions.apply(split_commissions)
complete_data.head(1)

,_id,type,date,url,summary,authors,parties,full_text,normalized_summary,normalized_summary_v2,...,doc_path,url_id,commissions,modified_laws,title,status,legislature,approval_date,da_summary,match_ratio
0,f02a7c92756ec00e5d112766d3d92214,iniciativas,2022-09-27,https://www.senado.gob.mx/64/gaceta_del_senado...,Propone actualizar las referencias al salario ...,[Noé Fernando Castañón Ramírez],[MC],C. Alejandro Armenta Mier\nPresidente de la Me...,proponer actualizar referencia salario mínimo ...,actualizar referencia salario mínimo unidad ac...,...,NaN,129274,"[Justicia, Estudios Legislativos]",Código Penal Federal<br>,Proyecto de decreto por el que se modifican di...,Pendiente,65,0000-00-00,Propone actualizar las referencias al salario ...,1.0


In [891]:
complete_data["modified_laws"] = complete_data.modified_laws.apply(split_laws)
complete_data.head(1)

,_id,type,date,url,summary,authors,parties,full_text,normalized_summary,normalized_summary_v2,...,doc_path,url_id,commissions,modified_laws,title,status,legislature,approval_date,da_summary,match_ratio
0,f02a7c92756ec00e5d112766d3d92214,iniciativas,2022-09-27,https://www.senado.gob.mx/64/gaceta_del_senado...,Propone actualizar las referencias al salario ...,[Noé Fernando Castañón Ramírez],[MC],C. Alejandro Armenta Mier\nPresidente de la Me...,proponer actualizar referencia salario mínimo ...,actualizar referencia salario mínimo unidad ac...,...,NaN,129274,"[Justicia, Estudios Legislativos]",[Código Penal Federal],Proyecto de decreto por el que se modifican di...,Pendiente,65,0000-00-00,Propone actualizar las referencias al salario ...,1.0


In [892]:
complete_data.status.unique()

array(['Pendiente', 'Aprobada', nan, 'Desechada'], dtype=object)

In [893]:
complete_data.loc[complete_data.approval_date == "0000-00-00", "approval_date"] = None

In [894]:
complete_data["approval_date"] = complete_data.approval_date.astype("datetime64[ns]")
complete_data["date"] = complete_data.date.astype("datetime64[ns]")

In [895]:
complete_data.loc[(~complete_data.title.isna()) & (complete_data.title.str.contains("<"))]

,_id,type,date,url,summary,authors,parties,full_text,normalized_summary,normalized_summary_v2,...,doc_path,url_id,commissions,modified_laws,title,status,legislature,approval_date,da_summary,match_ratio


In [896]:
complete_data.loc[~complete_data.legislature.isna(), "legislature"] = complete_data.loc[~complete_data.legislature.isna()].legislature.apply(int)

In [897]:
complete_data.loc[
    (complete_data.legislature.isna()) & (complete_data.date <= datetime(2021,8 ,31)), "legislature"
] = 64

complete_data.loc[
    (complete_data.legislature.isna()) & (complete_data.date >= datetime(2021, 9, 1)), "legislature"
] = 65

In [898]:
complete_data.legislature.isna().sum()

0

In [899]:
complete_data.dtypes

_id                                        object
type                                       object
date                               datetime64[ns]
url                                        object
summary                                    object
authors                                    object
parties                                    object
full_text                                  object
normalized_summary                         object
normalized_summary_v2                      object
summary_adj_noun                           object
summary_adj_only                           object
summary_nouns_only                         object
topics_normalized_text_12_model            object
doc_url                                    object
doc_path                                   object
url_id                                     object
commissions                                object
modified_laws                              object
title                                      object


## Iniciativas

In [900]:
initiatives = complete_data.loc[
    complete_data.type == "iniciativas"
]

In [901]:
initiatives = initiatives[[
    "_id",
    "date",
    "url",
    "title",
    "summary",
    "authors",
    "parties",
    "commissions",
    "modified_laws",
    "full_text",
    "doc_url",
    "status",
    "legislature",
    "approval_date"
]]
initiatives.head(1)

,_id,date,url,title,summary,authors,parties,commissions,modified_laws,full_text,doc_url,status,legislature,approval_date
0,f02a7c92756ec00e5d112766d3d92214,2022-09-27,https://www.senado.gob.mx/64/gaceta_del_senado...,Proyecto de decreto por el que se modifican di...,Propone actualizar las referencias al salario ...,[Noé Fernando Castañón Ramírez],[MC],"[Justicia, Estudios Legislativos]",[Código Penal Federal],C. Alejandro Armenta Mier\nPresidente de la Me...,NaN,Pendiente,65,NaT


In [903]:
initiatives = initiatives.replace({pd.NaT: None})

In [902]:
initiatives.shape

(6257, 14)

## Proposiciones

In [905]:
propositions = complete_data.loc[
    complete_data.type == "proposiciones"
]

In [906]:
propositions = propositions[[
    "_id",
    "date",
    "url",
    "title",
    "summary",
    "authors",
    "parties",
    "commissions",
    "full_text",
    "doc_url",
    "status",
    "legislature",
    "approval_date"
]]
propositions.head(1)

,_id,date,url,title,summary,authors,parties,commissions,full_text,doc_url,status,legislature,approval_date
30,24a9733ead619b0d4445ccff7f21cafa,2022-09-20,https://www.senado.gob.mx/64/gaceta_del_senado...,NaN,PRIMERO. El Senado de la República exhorta res...,[Ricardo Velázquez Meza],[MORENA],None,re na \nla esperanza ele México \nPROPOSICIÓN ...,https://infosen.senado.gob.mx/sgsp/gaceta/65/2...,NaN,65,NaT


In [907]:
propositions = propositions.replace({pd.NaT: None})

In [908]:
propositions.shape

(4686, 13)

## Unique publications

In [991]:
def collect_unique(column_values):
    return column_values.explode().unique().tolist()

In [992]:
def collect_list(column_values):
    return column_values.unique().tolist()

In [993]:
# create unique id by summary
complete_data["duplicate_id"] = complete_data.summary.apply(
    lambda s: hashlib.md5(s.encode("utf8")).hexdigest() 
)

In [994]:
complete_data.shape

(10943, 26)

In [995]:
duplicated_records = complete_data.duplicated(subset=["summary"], keep=False)
complete_data.loc[duplicated_records].shape

(1992, 26)

In [996]:
complete_data.loc[duplicated_records].duplicate_id.nunique()

400

In [997]:
10943 - 1992 + 400

9351

In [998]:
collect_unique(complete_data.iloc[:2].date)

[Timestamp('2022-09-27 00:00:00'), Timestamp('2022-09-20 00:00:00')]

In [999]:
complete_data[["authors", "parties", "date"]].isna().sum()

authors    0
parties    0
date       0
dtype: int64

In [1000]:
unique_data = complete_data.groupby(["duplicate_id", "type", "summary"]).agg({
    "_id": collect_list,
    "authors": collect_unique,
    "parties": collect_unique,
    "date": collect_unique
}).reset_index().rename(
    columns={
        "duplicate_id": "_id",
        "summary": "original_summary",
        "_id": "related_publications",
        "authors": "involved_authors",
        "parties": "involved_parties",
        "date": "related_dates"
    }
)
unique_data.head(1)

,_id,type,original_summary,related_publications,involved_authors,involved_parties,related_dates
0,00024466b3c4102e89e26260242f86f8,iniciativas,Propone establecer que los servicios de salud ...,[c38884d5c21fda9696168c56bc7f4f36],"[Elvia Marcela Mora Arellano, Eunice Renata Ro...",[PES],[2023-03-14 00:00:00]


In [1001]:
processed_data = complete_data[
    ["duplicate_id", "normalized_summary_v2", "summary_adj_noun", "summary_adj_only", "summary_nouns_only", "topics_normalized_text_12_model"]
].drop_duplicates(subset=["duplicate_id"])

In [1002]:
def get_processed_text_dict(row):
    return {
        "normalized": row.normalized_summary_v2,
        "adj_noun": row.summary_adj_noun,
        "adj_only": row.summary_adj_only,
        "noun_only": row.summary_nouns_only
    }

In [1003]:
def get_topics_list(topics):
    return [
        {
            "model_id": "normalized_text_12",
            "topics": topics
        }
    ]

In [1004]:
processed_data["processed_text"] = processed_data.apply(get_processed_text_dict, axis=1)
processed_data.head(1)

,duplicate_id,normalized_summary_v2,summary_adj_noun,summary_adj_only,summary_nouns_only,topics_normalized_text_12_model,processed_text
0,533fa82ded9de910c221b9678c2ae2c8,actualizar referencia salario mínimo unidad ac...,referencia salario mínimo unidad actualización...,salario mínimo vigente sujetos funcionario,referencia unidad actualización ciudad sanción,"[{'topic': 0, 'score': 0.14669232070446014}, {...",{'normalized': 'actualizar referencia salario ...


In [1005]:
processed_data["topics_distribution"] = processed_data.topics_normalized_text_12_model.apply(get_topics_list)
processed_data.head(1)

,duplicate_id,normalized_summary_v2,summary_adj_noun,summary_adj_only,summary_nouns_only,topics_normalized_text_12_model,processed_text,topics_distribution
0,533fa82ded9de910c221b9678c2ae2c8,actualizar referencia salario mínimo unidad ac...,referencia salario mínimo unidad actualización...,salario mínimo vigente sujetos funcionario,referencia unidad actualización ciudad sanción,"[{'topic': 0, 'score': 0.14669232070446014}, {...",{'normalized': 'actualizar referencia salario ...,"[{'model_id': 'normalized_text_12', 'topics': ..."


In [1006]:
processed_data = processed_data[["duplicate_id", "processed_text", "topics_distribution"]].rename(
    columns={"duplicate_id": "_id"}
)
processed_data.head()

,_id,processed_text,topics_distribution
0,533fa82ded9de910c221b9678c2ae2c8,{'normalized': 'actualizar referencia salario ...,"[{'model_id': 'normalized_text_12', 'topics': ..."
1,77751b8f40d06ad4b15e1ea4bd0dd6e0,{'normalized': 'educación impartido inculcar c...,"[{'model_id': 'normalized_text_12', 'topics': ..."
2,6ec27723072295469af9c65baca87e50,{'normalized': 'registro civil oficialía pobla...,"[{'model_id': 'normalized_text_12', 'topics': ..."
3,8e2a9c3a4b88b055a5acf7abb20e6746,{'normalized': 'atribuir bienestar función ela...,"[{'model_id': 'normalized_text_12', 'topics': ..."
4,b5f77c2a2f9d7e4f1e74a52bf22a5e79,{'normalized': 'desarrollo social fomentar cre...,"[{'model_id': 'normalized_text_12', 'topics': ..."


In [1007]:
unique_data_complete = unique_data.merge(
    processed_data,
    on="_id",
    how="inner"
)
unique_data_complete.shape

(9351, 9)

In [1008]:
unique_data_complete.loc[
    unique_data_complete.type == "iniciativas", "type"
] = "iniciativa"

unique_data_complete.loc[
    unique_data_complete.type == "proposiciones", "type"
] = "proposicion"

In [1009]:
unique_data_complete.isna().sum()

_id                     0
type                    0
original_summary        0
related_publications    0
involved_authors        0
involved_parties        0
related_dates           0
processed_text          0
topics_distribution     0
dtype: int64

## Guardar en mongo db

In [959]:
db.save_publications(
    publications=initiatives.to_dict(orient="records"),
    table_name="initiatives",
    conn=conn
)

In [960]:
db.save_publications(
    publications=propositions.to_dict(orient="records"),
    table_name="propositions",
    conn=conn
)

In [1010]:
db.save_publications(
    publications=unique_data_complete.to_dict(orient="records"),
    table_name="unique_publications",
    conn=conn
)

## Duplicados

_duplicados_

dup 1
- 30005511db35e9be759f068ebebbedb1
- 6e9be63e12069e55d41dac8caed513d7
- 55b0248f1e29bd2069b91eb331fafc13
- b598f597734516e0ffba796d436fd8a3
- 913160cde899abe55c159eee3932c123

dup 2
- 661009518fb4daa0ebfdd3500be57696
- ab75ed8a8b3e21a31d77866a3ff69e0c

In [564]:
duplicated_records = publications_df.duplicated(subset=["summary"], keep=False)
duplicates = publications_df[duplicated_records]
duplicates.shape

(1992, 17)

In [310]:
complete_duplicates = duplicates.merge(
    publications_datos_abiertos,
    on=["url_id"],
    how="left",
    suffixes=("_bot", "_json")
)
complete_duplicates.head(1)

,_id,type_bot,date_bot,url,summary_bot,authors_bot,parties,normalized_summary_v2,topics_normalized_text_12_model,url_id,date_json,legis_id,title,summary_json,authors_json,type_json,commissions,modified_laws
0,72765fb77b1a2e8bebdc2b53da7529ff,proposiciones,2022-09-20,https://www.senado.gob.mx/64/gaceta_del_senado...,PRIMERO. Se exhorta respetuosamente al Titular...,"[Martha Cecilia Márquez Alvarado, Joel Padilla...",[PT],titular infraestructura comunicación transport...,"[{'topic': 3, 'score': 0.14746767282485962}, {...",129120,2022-09-20,806166,Punto de acuerdo por el que se exhorta a la Gu...,PRIMERO. Se exhorta respetuosamente al Titular...,Martha Cecilia Márquez Alvarado (PT)<br>Joel P...,proposiciones,Comunicaciones y Transportes (Coordinadora) <br>,No Alpica


In [311]:
complete_duplicates["duplicate_id"] = complete_duplicates.summary_bot.apply(
    lambda s: hashlib.md5(s.encode("utf8")).hexdigest() 
)
complete_duplicates = complete_duplicates.sort_values(
    by="duplicate_id"
)

In [312]:
complete_duplicates.duplicate_id.nunique()

400

In [314]:
complete_duplicates.head()

,_id,type_bot,date_bot,url,summary_bot,authors_bot,parties,normalized_summary_v2,topics_normalized_text_12_model,url_id,date_json,legis_id,title,summary_json,authors_json,type_json,commissions,modified_laws,duplicate_id
960,544e7cd2ff04b92fc2f6ab206c29f9de,proposiciones,2020-06-17,https://www.senado.gob.mx/65/gaceta_del_senado...,ÚNICO. La Comisión Permanente del Congreso de ...,[Diputados],[PVEM],único autoridad materia salud tres órden gobie...,"[{'topic': 3, 'score': 0.14024245738983154}, {...",109038,2020-06-17,769223,Punto de acuerdo por el que se exhorta a la Se...,ÚNICO. La Comisión Permanente del Congreso de ...,Diputados (PVEM)<br>,proposiciones,"Segunda Comisión: Relaciones Exteriores, Defen...",No Alpica,000d612d3ad49b9bb49f6f1f4731c44e
802,9983c12bc258547fefb9062c3503f64d,proposiciones,2020-05-27,https://www.senado.gob.mx/65/gaceta_del_senado...,ÚNICO. La Comisión Permanente del Congreso de ...,[Diputados],[PVEM],único autoridad materia salud tres órden gobie...,"[{'topic': 3, 'score': 0.14023512601852417}, {...",108114,2020-05-27,769082,Punto de acuerdo por el que se exhorta a la Se...,ÚNICO. La Comisión Permanente del Congreso de ...,Diputados (PVEM)<br>,proposiciones,"Segunda Comisión: Relaciones Exteriores, Defen...",No Alpica,000d612d3ad49b9bb49f6f1f4731c44e
1478,436f20ca3a2dd9bc66549c5aa87969fa,proposiciones,2020-12-10,https://www.senado.gob.mx/65/gaceta_del_senado...,ÚNICO: El Senado de la República exhorta respe...,[Sasil De León Villard],[PES],único secretaria seguridad protección ciudadan...,"[{'topic': 1, 'score': 0.04422059655189514}, {...",114535,2020-12-10,769476,Punto de acuerdo por el que se exhorta a la Ad...,ÚNICO: El Senado de la República exhorta respe...,Sasil De León Villard (PES)<br>,proposiciones,Seguridad Pública (Coordinadora) <br>,No Alpica,00211c6e040c10b551e16b252a81d75d
1454,ec82af019c7e4aa351ae76decd081d08,proposiciones,2020-09-30,https://www.senado.gob.mx/65/gaceta_del_senado...,ÚNICO: El Senado de la República exhorta respe...,[Claudia Edith Anaya Mota],[PRI],único secretaria seguridad protección ciudadan...,"[{'topic': 1, 'score': 0.04422198608517647}, {...",112752,2020-09-30,769406,Punto de acuerdo por el que se exhorta a las S...,ÚNICO: El Senado de la República exhorta respe...,Claudia Edith Anaya Mota (PRI)<br>,proposiciones,Seguridad Pública (Coordinadora) <br>,No Alpica,00211c6e040c10b551e16b252a81d75d
1752,4c07b8795d09a2ebd091705c5a679799,proposiciones,2019-03-27,https://www.senado.gob.mx/65/gaceta_del_senado...,ÚNICO.- El Senado de la República exhorta resp...,[Jorge Carlos Ramírez Marín],[PRI],único agricultura desarrollo rural orientar co...,"[{'topic': 1, 'score': 0.09400620311498642}, {...",92089,2019-03-27,767838,Punto de acuerdo que exhorta a la Procuraduría...,ÚNICO.- El Senado de la República exhorta resp...,Jorge Carlos Ramírez Marín (PRI)<br>,proposiciones,Economía (Coordinadora) <br>,No Alpica,002f5aa7978ca57162b050b747e1e1e4


In [284]:
complete_duplicates.loc[
    complete_duplicates.legis_id.isna()
].shape

(47, 18)

In [530]:
agg_duplicates = complete_duplicates.groupby("duplicate_id").agg({
    "url": "count",
    "title": pd.Series.nunique,
    "summary_json": pd.Series.nunique,
    "summary_bot": pd.Series.nunique,
    "date_bot": pd.Series.nunique,
    "type_bot": pd.Series.nunique
}).rename(
    columns={
        "url": "num_records",
        "title": "unique_titles",
        "summary_json": "unique_json_summaries",
        "summary_bot": "unique_bot_summaries",
        "date_bot": "unique_dates",
        "type_bot": "unique_types"
    }
).reset_index().sort_values(
    by=["num_records"], ascending=False
).reset_index(drop=True)

In [531]:
agg_duplicates

,duplicate_id,num_records,unique_titles,unique_json_summaries,unique_bot_summaries,unique_dates,unique_types
0,898aceacd4e8faaf7f481c4c273c1e6b,103,103,1,1,1,1
1,520f21df5c4f4805b253fb068701ac80,100,100,1,1,1,1
2,7c26fd1651a3a5ff5bb748f6569ce11a,67,67,1,1,9,1
3,8644e60bca11459505ee365f918efbd5,45,45,1,1,1,1
4,7f8c57fe1c2860dfe0396e2a31d6f153,40,40,1,1,1,1
...,...,...,...,...,...,...,...
395,4419c8fe5c21412a68b2b16af5d5e68c,2,2,1,1,2,1
396,a2d35ef3cd7572bd029c09bfcbe1cfd3,2,1,1,1,2,1
397,424d502cd17597adfc89adbe149d31a9,2,2,1,1,2,1
398,416c9e08179d0174b33043a13e8900ff,2,2,1,1,2,1


In [339]:
agg_duplicates.loc[
    agg_duplicates.unique_titles == 0
]

,duplicate_id,num_records,unique_titles,unique_json_summaries,unique_bot_summaries,unique_dates
51,53f7730f350e22b6046b0987f33cdd65,7,0,0,1,5
106,79309938cf6c27b9022b539bc48aef67,4,0,0,1,3
119,0d6ada0141539f546d976c26c54dbecf,4,0,0,1,3
126,015e53ac7c96a876e459ef77ef954fd7,4,0,0,1,4
127,a45a81610e613f69d6941d38ce840eaf,4,0,0,1,2
187,06a5c720fcba7eaed27c98bd0821533f,3,0,0,1,3
202,6bc21855d87d1dea4b2c166bc693fd9d,3,0,0,1,3
218,2bc391a335c3eff0f568dcdf7acee2ae,2,0,0,1,2
221,bdb30848650463c43de17985f77aefbc,2,0,0,1,1
224,3348d146e23a76f8a291c6e433ba077b,2,0,0,1,2


In [330]:
agg_duplicates.loc[
    (agg_duplicates.unique_json_summaries != agg_duplicates.unique_bot_summaries)
    & (agg_duplicates.unique_json_summaries > 0)
]

,duplicate_id,num_records,unique_titles,unique_json_summaries,unique_bot_summaries


In [342]:
check_id = "53f7730f350e22b6046b0987f33cdd65"

In [343]:
complete_duplicates.loc[
    complete_duplicates.duplicate_id == check_id
].url.tolist()

['https://www.senado.gob.mx/65/gaceta_del_senado/documento/138016',
 'https://www.senado.gob.mx/65/gaceta_del_senado/documento/137460',
 'https://www.senado.gob.mx/65/gaceta_del_senado/documento/138517',
 'https://www.senado.gob.mx/65/gaceta_del_senado/documento/138522',
 'https://www.senado.gob.mx/65/gaceta_del_senado/documento/137804',
 'https://www.senado.gob.mx/65/gaceta_del_senado/documento/138530',
 'https://www.senado.gob.mx/65/gaceta_del_senado/documento/137805']

In [346]:
complete_duplicates.loc[
    complete_duplicates.duplicate_id == check_id
].iloc[0].summary_bot

'PRIMERO. El Senado de la República exhorta respetuosamente a los gobiernos de las entidades federativas y a la Comisión Federal de Electricidad, para que en el ámbito de sus atribuciones, lleven a cabo un convenio de colaboración para la aplicación de un subsidio para el próximo periodo de verano en apoyo a la economía de las familias mexicanas. SEGUNDO. El Senado de la República exhorta respetuosamente a la Secretaría de Energía, la Secretaría de Hacienda y Crédito Público y la Comisión Reguladora de Energía, para que en el ámbito de sus atribuciones, lleven a cabo mesas de trabajo a fin de analizar y discutir la metodología para el establecimiento de tarifas eléctricas.'

In [399]:
complete_duplicates.loc[
    complete_duplicates.duplicate_id == check_id
].url_id.tolist()

['138016', '137460', '138517', '138522', '137804', '138530', '137805']

In [467]:
def check_true_duplicates(duplicate_group):
    first_url = duplicate_group.url.iloc[0]
    first_url_id = duplicate_group.url_id.iloc[0]
    
    response = requests.get(first_url)
    bs = BeautifulSoup(response.content)
    
    meta_item = bs.find("meta")
    if meta_item is not None:
        attrs = meta_item.attrs
        if attrs.get("http-equiv", "") == "refresh":
            new_url = attrs["content"].split("url=")[-1]
            
            # get data from new url
            response = requests.get(new_url)
            bs = BeautifulSoup(response.content)
            
    # check links
    related_docs_ids = set()
    all_links = bs.find_all("a")
    for link in all_links:
        href = link.get("href")

        if href is not None and "gaceta_del_senado/documento" in href:
            related_docs_ids.add(href.split("/")[-1])
    
    # check for missing or extra ids
    if related_docs_ids:
        duplicate_url_ids = set(duplicate_group.url_id.tolist()) - {first_url_id}

        missing_ids = duplicate_url_ids.difference(related_docs_ids)
        extra_ids = related_docs_ids.difference(duplicate_url_ids)
    else:
        missing_ids = extra_ids = []
        
    return missing_ids, extra_ids

In [471]:
errors = {}
for i, row in tqdm(agg_duplicates.iterrows()):
    current_id = row.duplicate_id
    
    # get group
    duplicate_group = complete_duplicates.loc[
        complete_duplicates.duplicate_id == current_id
    ]
    
    try:
        # get urls that are not duplicates
        non_duplicate_ids, extra_duplicates = check_true_duplicates(duplicate_group)
    except Exception as ex:
        errors[current_id] = str(ex)
        continue
    
    # add data to original df
    agg_duplicates.loc[
        agg_duplicates.duplicate_id == current_id, "non_duplicate_ids"
    ] = ",".join(non_duplicate_ids)
    
    
    agg_duplicates.loc[
        agg_duplicates.duplicate_id == current_id, "extra_duplicate_ids"
    ] = ",".join(extra_duplicates)
    
    agg_duplicates.loc[
        agg_duplicates.duplicate_id == current_id, "all_duplicates"
    ] = len(non_duplicate_ids) == 0
    

400it [03:55,  1.70it/s]


In [472]:
agg_duplicates.groupby("all_duplicates").agg({
    "duplicate_id": "count"
})

,duplicate_id
all_duplicates,
False,5
True,395


In [479]:
agg_duplicates.loc[agg_duplicates.all_duplicates == False]

,duplicate_id,num_records,unique_titles,unique_json_summaries,unique_bot_summaries,unique_dates,non_duplicate_ids,all_duplicates,extra_duplicate_ids
0,898aceacd4e8faaf7f481c4c273c1e6b,103,103,1,1,1,"104150,104012,104104,104166,104068,104032,104153",False,"122839,122691"
1,520f21df5c4f4805b253fb068701ac80,100,100,1,1,1,"105993,105966,105970,106009,105940,105944,1059...",False,"103733,125091,103726,105505,103711,129021,1043..."
7,a1d95d2206584f196b460525a06312fa,31,31,1,1,1,112159,False,"111783,113237,113231,140111,111781,113240,1117..."
10,2b03cb425b54e3c944752dca958ee68e,20,20,1,1,1,"122749,122754,122743,122742,122748,122752,1227...",False,142428
12,08b5f3b2572ae7e3f27d9757f7f7e4fb,19,19,1,1,2,"111452,111442,111237,111443,111239,111235,1112...",False,"111783,113237,113231,140111,111781,112133,1121..."


In [501]:
agg_duplicates.loc[agg_duplicates.extra_duplicate_ids != ""]

,duplicate_id,num_records,unique_titles,unique_json_summaries,unique_bot_summaries,unique_dates,non_duplicate_ids,all_duplicates,extra_duplicate_ids
0,898aceacd4e8faaf7f481c4c273c1e6b,103,103,1,1,1,"104150,104012,104104,104166,104068,104032,104153",False,"122839,122691"
1,520f21df5c4f4805b253fb068701ac80,100,100,1,1,1,"105993,105966,105970,106009,105940,105944,1059...",False,"103733,125091,103726,105505,103711,129021,1043..."
3,8644e60bca11459505ee365f918efbd5,45,45,1,1,1,,True,"139158,140424,116796,114303,125362,140529,1224..."
5,8f22f8355588d46edeedd1a46adee9dd,40,40,1,1,1,,True,"140872,140819"
7,a1d95d2206584f196b460525a06312fa,31,31,1,1,1,112159,False,"111783,113237,113231,140111,111781,113240,1117..."
10,2b03cb425b54e3c944752dca958ee68e,20,20,1,1,1,"122749,122754,122743,122742,122748,122752,1227...",False,142428
12,08b5f3b2572ae7e3f27d9757f7f7e4fb,19,19,1,1,2,"111452,111442,111237,111443,111239,111235,1112...",False,"111783,113237,113231,140111,111781,112133,1121..."


In [505]:
for i, row in agg_duplicates.loc[agg_duplicates.extra_duplicate_ids != ""].iterrows():
    extra_ids = row.extra_duplicate_ids.split(",")
    
    other_dups = complete_duplicates.loc[complete_duplicates.url_id.isin(extra_ids)]
    if other_dups.shape[0] == 0:
        non_dups = publications_df.loc[publications_df.url_id.isin(extra_ids)]
        if non_dups.shape[0] == 0:
            print(f"[{row.duplicate_id}] {len(extra_ids)} does not exist")
        else:
            print(f"[{row.duplicate_id}] {len(extra_ids)} are not duplicates")
        
    

[898aceacd4e8faaf7f481c4c273c1e6b] 2 does not exist
[8644e60bca11459505ee365f918efbd5] 8 are not duplicates
[8f22f8355588d46edeedd1a46adee9dd] 2 does not exist
[2b03cb425b54e3c944752dca958ee68e] 1 does not exist


In [506]:
duplicate_group = complete_duplicates.loc[
    complete_duplicates.duplicate_id == "8644e60bca11459505ee365f918efbd5"
].sort_values(by="url")

In [507]:
duplicate_group.iloc[0].summary_bot

'Propone reformar el ordenamientos legales para garantizar la inclusión de pueblos y comunidades indígenas.'

In [515]:
extra_duplicates = agg_duplicates.loc[agg_duplicates.duplicate_id == "8644e60bca11459505ee365f918efbd5"].extra_duplicate_ids.iloc[0].split(",")
extra_duplicates

['139158',
 '140424',
 '116796',
 '114303',
 '125362',
 '140529',
 '122457',
 '125398']

In [518]:
publications_df.loc[publications_df.url_id.isin(extra_duplicates)]

,_id,type,date,url,summary,authors,parties,normalized_summary_v2,topics_normalized_text_12_model,url_id
3127,195713650a378cc8428fcccbbdf1fc8f,iniciativas,2021-11-30,https://www.senado.gob.mx/65/gaceta_del_senado...,Propone armonizar este ordenamiento jurídico e...,"[Susana Harp Iturribarría, María Celeste Sánch...",[MORENA],armonizar ordenamiento jurídico concordancia d...,"[{'topic': 0, 'score': 0.15135927498340607}, {...",122457
6986,1b467f675d6d65530cf79d5d0115501b,iniciativas,2020-12-03,https://www.senado.gob.mx/65/gaceta_del_senado...,Propone establecer en los ordenamientos legale...,"[Susana Harp Iturribarría, María Celeste Sánch...",[MORENA],ordenamiento legal vigente cuestión respeto pe...,"[{'topic': 1, 'score': 0.12333109974861145}, {...",114303
7445,65da8ba0c930ceade2c9b0a7d109421a,iniciativas,2021-04-29,https://www.senado.gob.mx/65/gaceta_del_senado...,Propone que se le de participación a las comun...,[Susana Harp Iturribarría],[MORENA],participación comunidad indígena igual manera ...,"[{'topic': 0, 'score': 0.17858749628067017}, {...",116796


In [464]:
complete_duplicates.loc[complete_duplicates.authors_bot]

set()

In [524]:
complete_duplicates.loc[complete_duplicates.authors_bot.apply(lambda authors: any("Blanca" in a for a in authors))]

,_id,type_bot,date_bot,url,summary_bot,authors_bot,parties,normalized_summary_v2,topics_normalized_text_12_model,url_id,date_json,legis_id,title,summary_json,authors_json,type_json,commissions,modified_laws,duplicate_id
671,de58bb0789db34ccca0b0aaf72bcfc74,iniciativas,2024-03-21,https://www.senado.gob.mx/65/gaceta_del_senado...,Propone establecer asambleas populares indígen...,[Blanca Estela Piña Gudiño],[MORENA],asamblea popular indígena integrado participar...,"[{'topic': 0, 'score': 0.2297646552324295}, {'...",141445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405fe52f916c409a5920250b7b6187e7
673,5644821ac553eb79dfa7c52bdab26722,iniciativas,2024-03-21,https://www.senado.gob.mx/65/gaceta_del_senado...,Propone establecer asambleas populares indígen...,[Blanca Estela Piña Gudiño],[MORENA],asamblea popular indígena integrado participar...,"[{'topic': 0, 'score': 0.22979110479354858}, {...",141446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405fe52f916c409a5920250b7b6187e7
1618,93c5d229096424da70efeefe691dbaea,proposiciones,2021-07-07,https://www.senado.gob.mx/65/gaceta_del_senado...,PRIMERO. La Comisión Permanente del H. Congres...,"[Daniel Gutiérrez Castorena, Blanca Estela Piñ...","[PAN, PRI, MORENA]",fiscalía general coordinación procuraduría pro...,"[{'topic': 0, 'score': 0.015456766821444035}, ...",118881,2021-07-07,769698,Punto de acuerdo por el que se exhorta al titu...,PRIMERO. La Comisión Permanente del H. Congres...,Daniel Gutiérrez Castorena (MORENA)<br>Blanca ...,proposiciones,"Primera Comisión: Gobernación, Puntos Constitu...",No Alpica,4b494881aecb8b77bf90ad7b32c6df1d
1449,3bd48a581d450624d2f22a1953b0a763,proposiciones,2020-09-23,https://www.senado.gob.mx/65/gaceta_del_senado...,"PRIMERO. El Senado de la República, exhorta re...","[Daniel Gutiérrez Castorena, Aníbal Ostoa Orte...","[MORENA, PT]",presupuesto cuenta público cámara diputado aná...,"[{'topic': 1, 'score': 0.2493317872285843}, {'...",112589,2020-09-23,769397,Punto de acuerdo por el que se exhorta a la Se...,"PRIMERO. El Senado de la República, exhorta re...",Daniel Gutiérrez Castorena (MORENA)<br>Aníbal ...,proposiciones,Educación (Coordinadora) <br>,No Alpica,7938bd6d8f0dc83a21ebb79134eec94c
83,1446d2b6ef7f882b205cb277ac78e505,proposiciones,2022-04-26,https://www.senado.gob.mx/65/gaceta_del_senado...,PRIMERO. El Senado de la República recomienda ...,"[Alejandra del Carmen León Gastélum, Bertha Al...","[PAN, PRI, PES, PVEM, MORENA, MC, Dip. Sin Grupo]",recomendar fiscalía procuraduría justicia 32 e...,"[{'topic': 0, 'score': 0.05394129455089569}, {...",125633,2022-04-26,805821,Punto de acuerdo por el que se exhorta al titu...,PRIMERO. El Senado de la República recomienda ...,Alejandra del Carmen León Gastélum (Dip. Sin G...,proposiciones,Para la Igualdad de Género (Coordinadora) <br>,No Alpica,b082b579a2c06e8310b7d09ba79cef7f
1861,90d495bed14bff08269217b5d6ca4d44,proposiciones,2019-06-26,https://www.senado.gob.mx/65/gaceta_del_senado...,ÚNICO. La Comisión Permanente del H. Congreso ...,"[Alejandra del Carmen León Gastélum, Bertha Al...","[MC, PAN, PRI, PT, MORENA]",único seguridad protección ciudadano guardia n...,"[{'topic': 3, 'score': 0.14846165478229523}, {...",96804,2019-06-26,768272,Punto de acuerdo en materia de violencia de gé...,ÚNICO. La Comisión Permanente del H. Congreso ...,Alejandra del Carmen León Gastélum (PT)<br>Ber...,proposiciones,"Primera Comisión: Gobernación, Puntos Constitu...",No Alpica,c4192e14512eb2c1e4af2564a6d0ddfc
1699,93259cc98de2fc215a14cdf4f9ae5db2,proposiciones,2018-10-31,https://www.senado.gob.mx/65/gaceta_del_senado...,PRIMERO. El Senado de la República reitera res...,[Blanca Estela Piña Gudiño],[MORENA],reiterar petición unidos contenido pronunciami...,"[{'topic': 0, 'score': 0.056098390370607376}, ...",85363,2018-10-31,767584,Punto de acuerdo relativo al bloqueo económico...,PRIMERO. El Senado de la República reitera res...,Blanca Estela Piña Gudiño (MORENA)<br>,proposiciones,Relaciones Exteriores América Latina y el Cari...,No Alpica,d704

In [525]:
agg_duplicates.loc[
    agg_duplicates.duplicate_id == "405fe52f916c409a5920250b7b6187e7"
]

,duplicate_id,num_records,unique_titles,unique_json_summaries,unique_bot_summaries,unique_dates,non_duplicate_ids,all_duplicates,extra_duplicate_ids
373,405fe52f916c409a5920250b7b6187e7,2,0,0,1,1,,True,


In [465]:
extra_ids

{'127135', '128583'}

In [529]:
agg_duplicates.loc[agg_duplicates.unique_dates > 1].shape

(322, 9)

In [532]:
agg_duplicates.loc[agg_duplicates.unique_types > 1].shape

(0, 7)